In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_pickle('data/1min_normalized.pkl')
df = df.reset_index()
df = df[pd.notnull(df).all(axis=1)]
df.head()

,index,date,pressure,temperature,humidity,so2,h2s,h2s_ref,captor,0,...,5,6,7,8,9,10,11,12,13,14
0,0,2016-05-23 08:30:00,0.0955967,1.004317,-1.618082,-0.432328,"[1.4231182466, 1.25451381326, 1.4231182466, 1....",-0.254843,1303,1.423118,...,1.254514,1.423118,1.254514,1.423118,1.423118,1.760327,1.254514,1.928932,1.591723,1.591723
1,1,2016-05-23 08:45:00,0.0955967,1.004317,-1.618082,-0.432328,"[1.25451381326, 1.4231182466, 1.76032711327, 1...",-0.254843,1303,1.254514,...,1.760327,1.423118,1.760327,1.591723,1.760327,1.423118,1.085909,1.423118,1.591723,1.760327
2,2,2016-05-23 09:00:00,0.0955967,1.004317,-1.618082,-0.432328,"[1.92893154661, 1.25451381326, 1.4231182466, 1...",-0.254843,1303,1.928932,...,1.591723,1.591723,1.760327,1.591723,1.423118,1.760327,1.760327,1.423118,1.760327,1.760327
3,3,2016-05-23 09:15:00,0.0955967,1.004317,-1.618082,-0.432328,"[1.25451381326, 1.25451381326, 1.59172267993, ...",-0.254843,1303,1.254514,...,1.591723,1.591723,1.760327,1.760327,1.591723,1.760327,1.591723,1.423118,1.591723,1.591723
4,4,2016-05-23 09:30:00,0.0955967,1.004317,-1.618082,-0.432328,"[1.59172267993, 1.76032711327, 1.59172267993, ...",-0.254843,1303,1.591723,...,1.591723,1.423118,1.254514,1.423118,1.760327,1.254514,1.085909,1.591723,1.591723,1.423118


In [26]:
samples_per_day = int(24*60/15) # nb_hours * nb_minutes_per_hours / sampling_size_in_minute
batch_size = samples_per_day * 4 # * 4 car sur 4 jours: [train, train, valid, test]
nb_batches = len(df) // batch_size
df = df[df['captor'] == '1304'] # 1304 is the longer timer series

In [59]:
df_train = pd.DataFrame(columns=['date', 'pressure', 'temperature','humidity', 'so2',\
                                 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 'batch'])
df_valid = pd.DataFrame(columns=['date', 'pressure', 'temperature','humidity', 'so2', \
                                 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 'batch'])
df_test = pd.DataFrame(columns=['date', 'pressure', 'temperature','humidity', 'so2',\
                                 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 'batch'])


i = 0
while i < nb_batches:
    k = 0
    for j in range(samples_per_day):
        cur = df.iloc[i * batch_size + k * samples_per_day + j, :]
        df_train.loc[len(df_train)] = [cur['date'], cur['pressure'], cur['temperature'], cur['humidity'],\
                       cur['so2'], cur[0], cur[1], cur[2], cur[3], cur[4], cur[5], cur[6], cur[7],\
                       cur[8], cur[9], cur[10], cur[11], cur[12], cur[13], cur[14], i]

    k += 1
    for j in range(samples_per_day):
        cur = df.iloc[i * batch_size + k * samples_per_day + j, :]
        df_train.loc[len(df_train)] = [cur['date'], cur['pressure'], cur['temperature'], cur['humidity'],\
                       cur['so2'], cur[0], cur[1], cur[2], cur[3], cur[4], cur[5], cur[6], cur[7],\
                       cur[8], cur[9], cur[10], cur[11], cur[12], cur[13], cur[14], i]
    k += 1
    for j in range(samples_per_day):
        cur = df.iloc[i * batch_size + k * samples_per_day + j, :]
        df_valid.loc[len(df_valid)] = [cur['date'], cur['pressure'], cur['temperature'], cur['humidity'],\
                       cur['so2'], cur[0], cur[1], cur[2], cur[3], cur[4], cur[5], cur[6], cur[7],\
                       cur[8], cur[9], cur[10], cur[11], cur[12], cur[13], cur[14], i]
    k += 1
    for j in range(samples_per_day):
        cur = df.iloc[i * batch_size + k * samples_per_day + j, :]
        df_test.loc[len(df_test)] = [cur['date'], cur['pressure'], cur['temperature'], cur['humidity'],\
                       cur['so2'], cur[0], cur[1], cur[2], cur[3], cur[4], cur[5], cur[6], cur[7],\
                       cur[8], cur[9], cur[10], cur[11], cur[12], cur[13], cur[14], i]
    i += 1
first_fill_idx = i * batch_size
remaining_samples = len(df) - (len(df_train) + len(df_valid) + len(df_test))
samples_nb_fill = remaining_samples // 4
k = 0

for j in range(samples_nb_fill):
        cur = df.iloc[first_fill_idx + k * samples_nb_fill + j, :]
        df_train.loc[len(df_train)] = [cur['date'], cur['pressure'], cur['temperature'], cur['humidity'],\
                       cur['so2'], cur[0], cur[1], cur[2], cur[3], cur[4], cur[5], cur[6], cur[7],\
                       cur[8], cur[9], cur[10], cur[11], cur[12], cur[13], cur[14], i]
for j in range(samples_nb_fill):
        cur = df.iloc[first_fill_idx + k * samples_nb_fill + j, :]
        df_train.loc[len(df_train)] = [cur['date'], cur['pressure'], cur['temperature'], cur['humidity'],\
                       cur['so2'], cur[0], cur[1], cur[2], cur[3], cur[4], cur[5], cur[6], cur[7],\
                       cur[8], cur[9], cur[10], cur[11], cur[12], cur[13], cur[14], i]
for j in range(samples_nb_fill):
        cur = df.iloc[first_fill_idx + k * samples_nb_fill + j, :]
        df_valid.loc[len(df_valid)] = [cur['date'], cur['pressure'], cur['temperature'], cur['humidity'],\
                       cur['so2'], cur[0], cur[1], cur[2], cur[3], cur[4], cur[5], cur[6], cur[7],\
                       cur[8], cur[9], cur[10], cur[11], cur[12], cur[13], cur[14], i]
for j in range(samples_nb_fill):
        cur = df.iloc[first_fill_idx + k * samples_nb_fill + j, :]
        df_test.loc[len(df_test)] = [cur['date'], cur['pressure'], cur['temperature'], cur['humidity'],\
                       cur['so2'], cur[0], cur[1], cur[2], cur[3], cur[4], cur[5], cur[6], cur[7],\
                       cur[8], cur[9], cur[10], cur[11], cur[12], cur[13], cur[14], i]
#new_batch_size = (len(df) - i) // 4
#j = 0
#while i + j < 


In [60]:
len(df_train), len(df_valid), len(df_test)

(5912, 2956, 2956)

(11520, 76, 11827)